In [ ]:
!pip install gdown
!gdown --id 1CowDEwmEOTZ38IhsEMfhxhPrWcbKZLwD
!gdown --id 1PInO2MlmZRa7Yp7fwBQsJBnWJFuJSZV1

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CowDEwmEOTZ38IhsEMfhxhPrWcbKZLwD
To: /content/truama.xlsx
100% 309k/309k [00:00<00:00, 67.5MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1PInO2MlmZRa7Yp7fwBQsJBnWJFuJSZV1
To: /content/bloodGlucose.xlsx
100% 244k/244k [00:00<00:00, 39.6MB/s]


In [ ]:
import pandas as pd

# Load data
dataset1 = pd.read_excel("bloodGlucose.xlsx")

# Separate static and time-series features
static_features_db1 = dataset1[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层', '工作状态', '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重', '身高', '孕前BMI']]
time_series_features_db1 = dataset1[['weight0', 'glu41', 'weight1', 'glu4b', 'weight2', 'glu4c', 'weight3', 'glu4d', 'weight4']]
target_db1 = dataset1['干预']

dataset2 = pd.read_excel('truama.xlsx')

# Separate Features and Target for Dataset 2
static_features_db2 = dataset2[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层',
                                 '工作状态', '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重',
                                 '身高', '孕前BMI', '意外怀孕', '助孕治疗', '已生孩子数分层',
                                 'ACE1', 'ACE2', 'ACE3', 'ACE4', 'ACE5', 'ACE6', 'ACE7', 'ACE8',
                                 'ACE9', 'ACE10', 'F1a', 'F2a', 'F3a', 'F4a', 'F5a', 'F6a',
                                 'F7a', 'F8a', 'F9a', 'F10a', 'F11a', 'F12a', 'F13a', 'F14a',
                                 'F15a', 'F16a', 'F17a', 'F18a', 'F19a']]
target_db2 = dataset2['干预']

In [ ]:
from sklearn.preprocessing import StandardScaler

# 创建标准化器
scaler_static_db1 = StandardScaler()
scaler_static_db2 = StandardScaler()

# 对训练数据进行标准化
static_features_db1[['年龄', '身高', '孕前BMI', '孕前体重']] = scaler_static_db1.fit_transform(static_features_db1[['年龄', '身高', '孕前BMI', '孕前体重']])
static_features_db2[['年龄', '身高', '孕前BMI', '孕前体重']] = scaler_static_db2.fit_transform(static_features_db2[['年龄', '身高', '孕前BMI', '孕前体重']])

# 保存标准化参数
mean_db1 = scaler_static_db1.mean_
scale_db1 = scaler_static_db1.scale_

mean_db2 = scaler_static_db2.mean_
scale_db2 = scaler_static_db2.scale_

print("Saved Mean DB1:", mean_db1)
print("Saved Scale DB1:", scale_db1)
print("Saved Mean DB2:", mean_db2)
print("Saved Scale DB2:", scale_db2)


Saved Mean DB1: [ 28.79748609 161.70849453  21.67057728  56.72085786]
Saved Scale DB1: [4.06999718 4.74138094 3.3589254  9.49922943]
Saved Mean DB2: [ 28.68893222 161.81691312  21.63959573  56.72005545]
Saved Scale DB2: [4.00608356 4.7094961  3.35030079 9.51160468]


<ipython-input-3-88688edc4f72>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_features_db1[['年龄', '身高', '孕前BMI', '孕前体重']] = scaler_static_db1.fit_transform(static_features_db1[['年龄', '身高', '孕前BMI', '孕前体重']])
<ipython-input-3-88688edc4f72>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_features_db2[['年龄', '身高', '孕前BMI', '孕前体重']] = scaler_static_db2.fit_transform(static_features_db2[['年龄', '身高', '孕前BMI', '孕前体重']])


In [ ]:
pip install dtaidistance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.7 MB/s eta 0:00:00


In [ ]:
# dtw
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import recall_score, precision_score, fbeta_score, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from dtaidistance import dtw
import numpy as np

# Function to fill missing values using DTW
def fill_nan_with_dtw(sequence, complete_sequences):
    """
    Replace np.nan in a sequence using DTW alignment with complete sequences.

    :param sequence: (array) The sequence containing np.nan
    :param complete_sequences: (list of arrays) List of complete sequences for alignment
    :return: (array) Sequence with np.nan replaced
    """
    # Identify missing value indices
    nan_indices = np.where(np.isnan(sequence))[0]

    # If no missing values, return the sequence as is
    if len(nan_indices) == 0:
        return sequence

    # Find the best-matching sequence using DTW
    best_match = None
    best_distance = float('inf')
    for seq in complete_sequences:
        # Compute DTW distance, ignoring np.nan in the sequence
        distance = dtw.distance(sequence[~np.isnan(sequence)], seq[:len(sequence)][~np.isnan(sequence)])
        if distance < best_distance:
            best_distance = distance
            best_match = seq

    # Fill missing values using the best-matching sequence
    if best_match is not None:
        filled_sequence = np.copy(sequence)
        for nan_index in nan_indices:
            filled_sequence[nan_index] = best_match[nan_index]
        return filled_sequence

    return sequence

# Add indices to static features
static_features_db1["index"] = static_features_db1.index
static_features_with_index = static_features_db1.to_numpy()
targets = target_db1.to_numpy()

static_features_db2["index"] = static_features_db2.index
static_features_with_index_db2 = static_features_db2.to_numpy()
targets_db2 = target_db2.to_numpy()

smote = SMOTE(random_state=42)
static_features_resampled, targets_resampled = smote.fit_resample(static_features_with_index, targets)
resampled_indices = static_features_resampled[:, -1].astype(int)  # Extract the index column
static_features_resampled = static_features_resampled[:, :-1]

static_features_resampled_db2, targets_resampled_db2 = smote.fit_resample(static_features_with_index_db2, targets_db2)
resampled_indices_db2 = static_features_resampled_db2[:, -1].astype(int)  # Extract the index column
static_features_resampled_db2 = static_features_resampled_db2[:, :-1]

glu_features = time_series_features_db1[['glu41', 'glu4b', 'glu4c', 'glu4d']].replace([' ', ''], np.nan).to_numpy()
weight_features = time_series_features_db1[['weight0', 'weight1', 'weight2', 'weight3', 'weight4']].replace([' ', ''], np.nan).to_numpy()

# DTW
# Extract complete sequences (no np.nan values)
complete_glu_sequences = [seq for seq in glu_features if not np.any(np.isnan(seq))]
complete_weight_sequences = [seq for seq in weight_features if not np.any(np.isnan(seq))]
# Fill missing values in glu_features
glu_features_filled = np.array([
    fill_nan_with_dtw(seq, complete_glu_sequences) for seq in glu_features
])
# Fill missing values in weight_features
weight_features_filled = np.array([
    fill_nan_with_dtw(seq, complete_weight_sequences) for seq in weight_features
])
glu_features_filled = np.nan_to_num(glu_features_filled.astype(np.float32), nan=0.0)
glu_features = glu_features_filled.reshape(glu_features_filled.shape[0], 4, 1)
weight_features = weight_features_filled.reshape(weight_features_filled.shape[0], 5, 1)


# Align time-series data using the resampled indices
glu_features_resampled = glu_features[resampled_indices]
weight_features_resampled = weight_features[resampled_indices]


# Split Data into Training and Testing Sets

# Split static, glu, weight, and targets into training and testing sets
X_static_train_db1, X_static_test_db1, X_glu_train, X_glu_test, X_weight_train, X_weight_test, y_train_db1, y_test_db1 = train_test_split(
    static_features_resampled, glu_features_resampled, weight_features_resampled, targets_resampled, test_size=0.1, random_state=42, stratify=targets_resampled
)

X_static_train_db2, X_static_test_db2, y_train_db2, y_test_db2 = train_test_split(
    static_features_resampled_db2, targets_resampled_db2, test_size=0.1, random_state=42, stratify=targets_resampled_db2
)

# Print data shapes to verify
print("Dataset 1 Static Training Shape:", X_static_train_db1.shape)
print("Dataset 1 Glu Training Shape:", X_glu_train.shape)
print("Dataset 1 Weight Training Shape:", X_weight_train.shape)
print("Dataset 2 Static Training Shape:", X_static_train_db2.shape)
class CustomDataset(Dataset):
    def __init__(self, static_data, glu_data, weight_data, target):
        # Handle pandas DataFrame or NumPy array
        if isinstance(static_data, pd.DataFrame):
            self.static_data = torch.tensor(static_data.to_numpy(), dtype=torch.float32)
        else:
            self.static_data = torch.tensor(static_data, dtype=torch.float32)

        self.glu_data = torch.tensor(glu_data, dtype=torch.float32)  # glu_data is already NumPy
        self.weight_data = torch.tensor(weight_data, dtype=torch.float32)  # weight_data is already NumPy
        self.target = torch.tensor(target, dtype=torch.float32)  # Convert to tensor

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return {
            "static": self.static_data[idx],
            "glu": self.glu_data[idx],
            "weight": self.weight_data[idx],
            "target": self.target[idx],
        }

<ipython-input-5-99ef90a92b84>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_features_db1["index"] = static_features_db1.index
<ipython-input-5-99ef90a92b84>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_features_db2["index"] = static_features_db2.index
<ipython-input-5-99ef90a92b84>:66: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the 

Dataset 1 Static Training Shape: (1584, 12)
Dataset 1 Glu Training Shape: (1584, 4, 1)
Dataset 1 Weight Training Shape: (1584, 5, 1)
Dataset 2 Static Training Shape: (1548, 44)


In [ ]:
# 正常Masking版本
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import recall_score, precision_score, fbeta_score, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE

# Add indices to static features
static_features_db1["index"] = static_features_db1.index
static_features_with_index = static_features_db1.to_numpy()
targets = target_db1.to_numpy()

static_features_db2["index"] = static_features_db2.index
static_features_with_index_db2 = static_features_db2.to_numpy()
targets_db2 = target_db2.to_numpy()

smote = SMOTE(random_state=42)
static_features_resampled, targets_resampled = smote.fit_resample(static_features_with_index, targets)
resampled_indices = static_features_resampled[:, -1].astype(int)  # Extract the index column
static_features_resampled = static_features_resampled[:, :-1]

static_features_resampled_db2, targets_resampled_db2 = smote.fit_resample(static_features_with_index_db2, targets_db2)
resampled_indices_db2 = static_features_resampled_db2[:, -1].astype(int)  # Extract the index column
static_features_resampled_db2 = static_features_resampled_db2[:, :-1]

glu_features = time_series_features_db1[['glu41', 'glu4b', 'glu4c', 'glu4d']].replace([' ', ''], np.nan).to_numpy()
weight_features = time_series_features_db1[['weight0', 'weight1', 'weight2', 'weight3', 'weight4']].replace([' ', ''], np.nan).to_numpy()


glu_features = np.nan_to_num(glu_features.astype(np.float32), nan=0.0)
weight_features = np.nan_to_num(weight_features.astype(np.float32), nan=0.0)


# Reshape to (samples, time_steps, features)
glu_features = glu_features.reshape(glu_features.shape[0], 4, 1)
weight_features = weight_features.reshape(weight_features.shape[0], 5, 1)

# Align time-series data using the resampled indices
glu_features_resampled = glu_features[resampled_indices]
weight_features_resampled = weight_features[resampled_indices]


# Split Data into Training and Testing Sets

# Split static, glu, weight, and targets into training and testing sets
X_static_train_db1, X_static_test_db1, X_glu_train, X_glu_test, X_weight_train, X_weight_test, y_train_db1, y_test_db1 = train_test_split(
    static_features_resampled, glu_features_resampled, weight_features_resampled, targets_resampled, test_size=0.1, random_state=42, stratify=targets_resampled
)
X_static_train_db2, X_static_test_db2, y_train_db2, y_test_db2 = train_test_split(
    static_features_resampled_db2, targets_resampled_db2, test_size=0.1, random_state=42, stratify=targets_resampled_db2
)

# Print data shapes to verify
print("Dataset 1 Static Training Shape:", X_static_train_db1.shape)
print("Dataset 1 Glu Training Shape:", X_glu_train.shape)
print("Dataset 1 Weight Training Shape:", X_weight_train.shape)
print("Dataset 2 Static Training Shape:", X_static_train_db2.shape)
class CustomDataset(Dataset):
    def __init__(self, static_data, glu_data, weight_data, target):
        # Handle pandas DataFrame or NumPy array
        if isinstance(static_data, pd.DataFrame):
            self.static_data = torch.tensor(static_data.to_numpy(), dtype=torch.float32)
        else:
            self.static_data = torch.tensor(static_data, dtype=torch.float32)

        self.glu_data = torch.tensor(glu_data, dtype=torch.float32)  # glu_data is already NumPy
        self.weight_data = torch.tensor(weight_data, dtype=torch.float32)  # weight_data is already NumPy
        self.target = torch.tensor(target, dtype=torch.float32)  # Convert to tensor

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return {
            "static": self.static_data[idx],
            "glu": self.glu_data[idx],
            "weight": self.weight_data[idx],
            "target": self.target[idx],
        }

Dataset 1 Static Training Shape: (1584, 12)
Dataset 1 Glu Training Shape: (1584, 4, 1)
Dataset 1 Weight Training Shape: (1584, 5, 1)
Dataset 2 Static Training Shape: (1548, 44)


<ipython-input-55-366d10ee84ba>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_features_db1["index"] = static_features_db1.index
<ipython-input-55-366d10ee84ba>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_features_db2["index"] = static_features_db2.index
<ipython-input-55-366d10ee84ba>:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to t

In [ ]:
# dynamic threshold & 无scheduler
from sklearn.metrics import precision_recall_curve
import numpy as np

def find_best_threshold(outputs, targets, beta=2, min_threshold=0.1):
    precision, recall, thresholds = precision_recall_curve(targets, outputs)

    # Compute F-beta score for thresholds
    fbeta_scores = (1 + beta**2) * (precision[:-1] * recall[:-1]) / ((beta**2 * precision[:-1]) + recall[:-1] + 1e-5)

    # Filter out thresholds that are too low
    valid_idx = thresholds >= min_threshold
    thresholds = thresholds[valid_idx]
    fbeta_scores = fbeta_scores[valid_idx]

    if len(fbeta_scores) == 0:
        # Fallback in case all thresholds are filtered out
        return min_threshold, 0.0

    best_idx = np.argmax(fbeta_scores)
    return thresholds[best_idx], fbeta_scores[best_idx]


train_dataset_db1 = CustomDataset(X_static_train_db1, X_glu_train, X_weight_train, y_train_db1)
test_dataset_db1 = CustomDataset(X_static_test_db1, X_glu_test, X_weight_test, y_test_db1)

train_loader_db1 = DataLoader(train_dataset_db1, batch_size=32, shuffle=True)
test_loader_db1 = DataLoader(test_dataset_db1, batch_size=32, shuffle=False)

# Define the Multi-Input Model
class MultiInputModel(nn.Module):
    def __init__(self, static_input_dim, glu_input_dim, weight_input_dim):
        super(MultiInputModel, self).__init__()

        # Static feature branch
        self.static_fc = nn.Sequential(
            nn.Linear(static_input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for glu with attention and masking
        self.glu_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.glu_attention = nn.Sequential(
            nn.Linear(64, 1),  # Attention score for each time step
            nn.Softmax(dim=1)  # Normalize scores across valid time steps
        )
        self.glu_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for weight with attention and masking
        self.weight_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.weight_attention = nn.Sequential(
            nn.Linear(64, 1),
            nn.Softmax(dim=1)
        )
        self.weight_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Combined high-level features
        self.combined_fc = nn.Sequential(
            nn.Linear(32 + 32 + 32, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, static_input, glu_input, weight_input):
        # Static feature branch
        static_out = self.static_fc(static_input)

        # Glu branch with masking and attention
        glu_mask = (glu_input.sum(dim=-1) != 0).float()  # Mask for valid time steps
        glu_out, _ = self.glu_lstm(glu_input)  # Output shape: (batch_size, seq_len, hidden_size)

        attention_scores_glu = self.glu_attention(glu_out).squeeze(-1)  # Attention scores: (batch_size, seq_len)
        attention_scores_glu = attention_scores_glu * glu_mask  # Apply mask to attention scores
        attention_weights_glu = attention_scores_glu / (attention_scores_glu.sum(dim=1, keepdim=True) + 1e-8)  # Normalize

        glu_out = torch.bmm(attention_weights_glu.unsqueeze(1), glu_out).squeeze(1)  # Weighted sum: (batch_size, hidden_size)
        glu_out = self.glu_fc(glu_out)

        # Weight branch with masking and attention
        weight_mask = (weight_input.sum(dim=-1) != 0).float()  # Mask for valid time steps
        weight_out, _ = self.weight_lstm(weight_input)

        attention_scores_weight = self.weight_attention(weight_out).squeeze(-1)  # Attention scores: (batch_size, seq_len)
        attention_scores_weight = attention_scores_weight * weight_mask  # Apply mask to attention scores
        attention_weights_weight = attention_scores_weight / (attention_scores_weight.sum(dim=1, keepdim=True) + 1e-8)  # Normalize

        weight_out = torch.bmm(attention_weights_weight.unsqueeze(1), weight_out).squeeze(1)  # Weighted sum: (batch_size, hidden_size)
        weight_out = self.weight_fc(weight_out)

        # Combine all branches
        combined = torch.cat([static_out, glu_out, weight_out], dim=1)
        output = self.combined_fc(combined)
        return output



# Initialize the Model, Loss, and Optimizer
static_input_dim = X_static_train_db1.shape[1]
glu_input_dim = X_glu_train.shape[1]
weight_input_dim = X_weight_train.shape[1]
print(static)
model = MultiInputModel(static_input_dim, glu_input_dim, weight_input_dim)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.0005)
# Training Model
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=50):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch in train_loader:
            static_input = batch["static"]
            glu_input = batch["glu"]
            weight_input = batch["weight"]
            target = batch["target"].unsqueeze(1)

            # Forward pass
            outputs = model(static_input, glu_input, weight_input)
            loss = criterion(outputs, target)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Evaluate model and find the best threshold
        outputs, targets, best_threshold, recall, precision, f2_score, accuracy = evaluate_model(model, test_loader)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, "
              f"Best Threshold: {best_threshold:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, "
              f"F2 Score: {f2_score:.4f}, Accuracy: {accuracy:.4f}")

def calculate_metrics(outputs, targets, threshold=0.5):
    predictions = (outputs > threshold).astype(int)
    targets = targets.astype(int)
    recall = recall_score(targets, predictions)
    precision = precision_score(targets, predictions)
    f2_score = fbeta_score(targets, predictions, beta=2)
    accuracy = accuracy_score(targets, predictions)
    return recall, precision, f2_score, accuracy

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for batch in test_loader:
            static_input = batch["static"]
            glu_input = batch["glu"]
            weight_input = batch["weight"]
            target = batch["target"]

            outputs = model(static_input, glu_input, weight_input)
            all_outputs.append(outputs.squeeze().numpy())
            all_targets.append(target.numpy())

    outputs = np.concatenate(all_outputs)
    targets = np.concatenate(all_targets)

    # Find the best dynamic threshold
    best_threshold, best_fbeta = find_best_threshold(outputs, targets, beta=2)

    # Apply the best threshold
    predictions = (outputs > best_threshold).astype(int)
    recall = recall_score(targets, predictions)
    precision = precision_score(targets, predictions)
    f2_score = fbeta_score(targets, predictions, beta=2)
    accuracy = accuracy_score(targets, predictions)

    return outputs, targets, best_threshold, recall, precision, f2_score, accuracy


train_model(model, train_loader_db1, test_loader_db1, criterion, optimizer)

Epoch [1/50], Loss: 0.6950, Best Threshold: 0.4575, Recall: 0.9773, Precision: 0.5309, F2 Score: 0.8366, Accuracy: 0.5568
Epoch [2/50], Loss: 0.6777, Best Threshold: 0.3426, Recall: 0.9773, Precision: 0.5276, F2 Score: 0.8350, Accuracy: 0.5511
Epoch [3/50], Loss: 0.6287, Best Threshold: 0.1805, Recall: 0.9773, Precision: 0.5409, F2 Score: 0.8415, Accuracy: 0.5739
Epoch [4/50], Loss: 0.6107, Best Threshold: 0.2142, Recall: 0.9886, Precision: 0.5273, F2 Score: 0.8414, Accuracy: 0.5511
Epoch [5/50], Loss: 0.5800, Best Threshold: 0.1587, Recall: 0.9886, Precision: 0.5273, F2 Score: 0.8414, Accuracy: 0.5511
Epoch [6/50], Loss: 0.5505, Best Threshold: 0.1592, Recall: 0.9886, Precision: 0.5305, F2 Score: 0.8430, Accuracy: 0.5568
Epoch [7/50], Loss: 0.5359, Best Threshold: 0.1242, Recall: 0.9886, Precision: 0.5337, F2 Score: 0.8447, Accuracy: 0.5625
Epoch [8/50], Loss: 0.5141, Best Threshold: 0.1420, Recall: 0.9886, Precision: 0.5241, F2 Score: 0.8398, Accuracy: 0.5455
Epoch [9/50], Loss: 0.50

In [ ]:
# Normal Version
from torch.optim.lr_scheduler import ReduceLROnPlateau
train_dataset_db1 = CustomDataset(X_static_train_db1, X_glu_train, X_weight_train, y_train_db1)
test_dataset_db1 = CustomDataset(X_static_test_db1, X_glu_test, X_weight_test, y_test_db1)

train_loader_db1 = DataLoader(train_dataset_db1, batch_size=32, shuffle=True)
test_loader_db1 = DataLoader(test_dataset_db1, batch_size=32, shuffle=False)

# Define the Multi-Input Model
class MultiInputModel(nn.Module):
    def __init__(self, static_input_dim, glu_input_dim, weight_input_dim):
        super(MultiInputModel, self).__init__()

        # Static feature branch
        self.static_fc = nn.Sequential(
            nn.Linear(static_input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for glu with attention and masking
        self.glu_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.glu_attention = nn.Sequential(
            nn.Linear(64, 1),  # Attention score for each time step
            nn.Softmax(dim=1)  # Normalize scores across valid time steps
        )
        self.glu_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for weight with attention and masking
        self.weight_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.weight_attention = nn.Sequential(
            nn.Linear(64, 1),
            nn.Softmax(dim=1)
        )
        self.weight_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Combined high-level features
        self.combined_fc = nn.Sequential(
            nn.Linear(32 + 32 + 32, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, static_input, glu_input, weight_input):
        # Static feature branch
        static_out = self.static_fc(static_input)

        # Glu branch with masking and attention
        glu_mask = (glu_input.sum(dim=-1) != 0).float()  # Mask for valid time steps
        glu_out, _ = self.glu_lstm(glu_input)  # Output shape: (batch_size, seq_len, hidden_size)

        attention_scores_glu = self.glu_attention(glu_out).squeeze(-1)  # Attention scores: (batch_size, seq_len)
        attention_scores_glu = attention_scores_glu * glu_mask  # Apply mask to attention scores
        attention_weights_glu = attention_scores_glu / (attention_scores_glu.sum(dim=1, keepdim=True) + 1e-8)  # Normalize

        glu_out = torch.bmm(attention_weights_glu.unsqueeze(1), glu_out).squeeze(1)  # Weighted sum: (batch_size, hidden_size)
        glu_out = self.glu_fc(glu_out)

        # Weight branch with masking and attention
        weight_mask = (weight_input.sum(dim=-1) != 0).float()  # Mask for valid time steps
        weight_out, _ = self.weight_lstm(weight_input)

        attention_scores_weight = self.weight_attention(weight_out).squeeze(-1)  # Attention scores: (batch_size, seq_len)
        attention_scores_weight = attention_scores_weight * weight_mask  # Apply mask to attention scores
        attention_weights_weight = attention_scores_weight / (attention_scores_weight.sum(dim=1, keepdim=True) + 1e-8)  # Normalize

        weight_out = torch.bmm(attention_weights_weight.unsqueeze(1), weight_out).squeeze(1)  # Weighted sum: (batch_size, hidden_size)
        weight_out = self.weight_fc(weight_out)

        # Combine all branches
        combined = torch.cat([static_out, glu_out, weight_out], dim=1)
        output = self.combined_fc(combined)
        return output


# weighted loss function
num_positive = target_db1.sum()
num_negative = len(target_db1) - num_positive
pos_weight_value = num_negative / num_positive
pos_weight = torch.tensor([pos_weight_value], dtype=torch.float32)

# Initialize the Model, Loss, and Optimizer
static_input_dim = X_static_train_db1.shape[1]
glu_input_dim = X_glu_train.shape[1]
weight_input_dim = X_weight_train.shape[1]

model = MultiInputModel(static_input_dim, glu_input_dim, weight_input_dim)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

# Training Model
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=50, threshold=0.5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch in train_loader:
            static_input = batch["static"]
            glu_input = batch["glu"]
            weight_input = batch["weight"]
            target = batch["target"].unsqueeze(1)  # Ensure target is of shape (batch_size, 1)

            # Forward pass
            outputs = model(static_input, glu_input, weight_input)
            outputs = outputs.clamp(min=1e-7, max=1 - 1e-7)  # Clamp outputs for numerical stability

            # Compute loss
            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Evaluate on the test set
        _, _, recall, precision, f2_score, accuracy = evaluate_model(model, test_loader, threshold)
        scheduler.step(f2_score)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, "
              f" Recall: {recall:.4f}, Precision: {precision:.4f}, "
              f"F2 Score: {f2_score:.4f}, Accuracy: {accuracy:.4f}")


def calculate_metrics(outputs, targets, threshold=0.5):
    predictions = (outputs > threshold).astype(int)
    targets = targets.astype(int)
    recall = recall_score(targets, predictions)
    precision = precision_score(targets, predictions)
    f2_score = fbeta_score(targets, predictions, beta=2)
    accuracy = accuracy_score(targets, predictions)
    return recall, precision, f2_score, accuracy

# Evaluation
def evaluate_model(model, test_loader, threshold=0.5):
    model.eval()
    all_outputs = []
    all_targets = []
    with torch.no_grad():
        for batch in test_loader:
            static_input = batch["static"]
            glu_input = batch["glu"]
            weight_input = batch["weight"]
            target = batch["target"]

            outputs = model(static_input, glu_input, weight_input)
            all_outputs.append(outputs.squeeze().numpy())
            all_targets.append(target.numpy())

    outputs = np.concatenate(all_outputs)
    targets = np.concatenate(all_targets)
    recall, precision, f2_score, accuracy = calculate_metrics(outputs, targets, threshold)
    return outputs, targets, recall, precision, f2_score, accuracy

train_model(model, train_loader_db1, test_loader_db1, criterion, optimizer)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/50], Loss: 0.6927,  Recall: 0.1705, Precision: 0.6250, F2 Score: 0.1995, Accuracy: 0.5341
Epoch [2/50], Loss: 0.6725,  Recall: 0.1818, Precision: 0.7273, F2 Score: 0.2139, Accuracy: 0.5568
Epoch [3/50], Loss: 0.6414,  Recall: 0.3523, Precision: 0.7949, F2 Score: 0.3964, Accuracy: 0.6307
Epoch [4/50], Loss: 0.5986,  Recall: 0.6250, Precision: 0.6548, F2 Score: 0.6307, Accuracy: 0.6477
Epoch [5/50], Loss: 0.5684,  Recall: 0.6932, Precision: 0.6354, F2 Score: 0.6808, Accuracy: 0.6477
Epoch [6/50], Loss: 0.5467,  Recall: 0.4432, Precision: 0.7647, F2 Score: 0.4839, Accuracy: 0.6534
Epoch [7/50], Loss: 0.5117,  Recall: 0.6932, Precision: 0.6630, F2 Score: 0.6869, Accuracy: 0.6705
Epoch [8/50], Loss: 0.4970,  Recall: 0.7955, Precision: 0.6364, F2 Score: 0.7576, Accuracy: 0.6705
Epoch [9/50], Loss: 0.4793,  Recall: 0.5795, Precision: 0.6986, F2 Score: 0.6000, Accuracy: 0.6648
Epoch [10/50], Loss: 0.4508,  Recall: 0.6136, Precision: 0.7105, F2 Score: 0.6308, Accuracy: 0.6818
Epoch [11

In [ ]:
# weighted loss function + focus loss
from torch.optim.lr_scheduler import ReduceLROnPlateau
train_dataset_db1 = CustomDataset(X_static_train_db1, X_glu_train, X_weight_train, y_train_db1)
test_dataset_db1 = CustomDataset(X_static_test_db1, X_glu_test, X_weight_test, y_test_db1)

train_loader_db1 = DataLoader(train_dataset_db1, batch_size=32, shuffle=True)
test_loader_db1 = DataLoader(test_dataset_db1, batch_size=32, shuffle=False)

# Define the Multi-Input Model
class MultiInputModel(nn.Module):
    def __init__(self, static_input_dim, glu_input_dim, weight_input_dim):
        super(MultiInputModel, self).__init__()

        # Static feature branch
        self.static_fc = nn.Sequential(
            nn.Linear(static_input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for glu with attention and masking
        self.glu_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.glu_attention = nn.Sequential(
            nn.Linear(64, 1),  # Attention score for each time step
            nn.Softmax(dim=1)  # Normalize scores across valid time steps
        )
        self.glu_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for weight with attention and masking
        self.weight_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.weight_attention = nn.Sequential(
            nn.Linear(64, 1),
            nn.Softmax(dim=1)
        )
        self.weight_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Combined high-level features
        self.combined_fc = nn.Sequential(
            nn.Linear(32 + 32 + 32, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, static_input, glu_input, weight_input):
        # Static feature branch
        static_out = self.static_fc(static_input)

        # Glu branch with masking and attention
        glu_mask = (glu_input.sum(dim=-1) != 0).float()  # Mask for valid time steps
        glu_out, _ = self.glu_lstm(glu_input)  # Output shape: (batch_size, seq_len, hidden_size)

        attention_scores_glu = self.glu_attention(glu_out).squeeze(-1)  # Attention scores: (batch_size, seq_len)
        attention_scores_glu = attention_scores_glu * glu_mask  # Apply mask to attention scores
        attention_weights_glu = attention_scores_glu / (attention_scores_glu.sum(dim=1, keepdim=True) + 1e-8)  # Normalize

        glu_out = torch.bmm(attention_weights_glu.unsqueeze(1), glu_out).squeeze(1)  # Weighted sum: (batch_size, hidden_size)
        glu_out = self.glu_fc(glu_out)

        # Weight branch with masking and attention
        weight_mask = (weight_input.sum(dim=-1) != 0).float()  # Mask for valid time steps
        weight_out, _ = self.weight_lstm(weight_input)

        attention_scores_weight = self.weight_attention(weight_out).squeeze(-1)  # Attention scores: (batch_size, seq_len)
        attention_scores_weight = attention_scores_weight * weight_mask  # Apply mask to attention scores
        attention_weights_weight = attention_scores_weight / (attention_scores_weight.sum(dim=1, keepdim=True) + 1e-8)  # Normalize

        weight_out = torch.bmm(attention_weights_weight.unsqueeze(1), weight_out).squeeze(1)  # Weighted sum: (batch_size, hidden_size)
        weight_out = self.weight_fc(weight_out)

        # Combine all branches
        combined = torch.cat([static_out, glu_out, weight_out], dim=1)
        output = self.combined_fc(combined)
        return output



class WeightedFocalLoss(nn.Module):
    def __init__(self, pos_weight, gamma=2):
        super(WeightedFocalLoss, self).__init__()
        self.pos_weight = pos_weight
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.BCELoss(weight=(targets * self.pos_weight + (1 - targets)))(inputs, targets)
        pt = torch.exp(-BCE_loss)  # pt is the probability of the correct class
        focal_loss = (1 - pt) ** self.gamma * BCE_loss
        return torch.mean(focal_loss)

# Initialize the Model, Loss, and Optimizer
static_input_dim = X_static_train_db1.shape[1]
glu_input_dim = X_glu_train.shape[1]
weight_input_dim = X_weight_train.shape[1]

model = MultiInputModel(static_input_dim, glu_input_dim, weight_input_dim)
num_positive = target_db1.sum()
num_negative = len(target_db1) - num_positive
pos_weight = num_negative / num_positive
criterion = WeightedFocalLoss(pos_weight=pos_weight, gamma=2)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

# Training Model
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=50, threshold=0.5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch in train_loader:
            static_input = batch["static"]
            glu_input = batch["glu"]
            weight_input = batch["weight"]
            target = batch["target"].unsqueeze(1)  # Ensure target is of shape (batch_size, 1)

            # Forward pass
            outputs = model(static_input, glu_input, weight_input)
            outputs = outputs.clamp(min=1e-7, max=1 - 1e-7)  # Clamp outputs for numerical stability

            # Compute loss
            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Evaluate on the test set
        _, _, recall, precision, f2_score, accuracy = evaluate_model(model, test_loader, threshold)
        scheduler.step(f2_score)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, "
              f" Recall: {recall:.4f}, Precision: {precision:.4f}, "
              f"F2 Score: {f2_score:.4f}, Accuracy: {accuracy:.4f}")


def calculate_metrics(outputs, targets, threshold=0.5):
    predictions = (outputs > threshold).astype(int)
    targets = targets.astype(int)
    recall = recall_score(targets, predictions)
    precision = precision_score(targets, predictions)
    f2_score = fbeta_score(targets, predictions, beta=2)
    accuracy = accuracy_score(targets, predictions)
    return recall, precision, f2_score, accuracy

# Evaluation
def evaluate_model(model, test_loader, threshold=0.5):
    model.eval()
    all_outputs = []
    all_targets = []
    with torch.no_grad():
        for batch in test_loader:
            static_input = batch["static"]
            glu_input = batch["glu"]
            weight_input = batch["weight"]
            target = batch["target"]

            outputs = model(static_input, glu_input, weight_input)
            all_outputs.append(outputs.squeeze().numpy())
            all_targets.append(target.numpy())

    outputs = np.concatenate(all_outputs)
    targets = np.concatenate(all_targets)
    recall, precision, f2_score, accuracy = calculate_metrics(outputs, targets, threshold)
    return outputs, targets, recall, precision, f2_score, accuracy

train_model(model, train_loader_db1, test_loader_db1, criterion, optimizer)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/50], Loss: 0.5550,  Recall: 1.0000, Precision: 0.5000, F2 Score: 0.8333, Accuracy: 0.5000
Epoch [2/50], Loss: 0.4552,  Recall: 1.0000, Precision: 0.5000, F2 Score: 0.8333, Accuracy: 0.5000
Epoch [3/50], Loss: 0.4141,  Recall: 1.0000, Precision: 0.5000, F2 Score: 0.8333, Accuracy: 0.5000
Epoch [4/50], Loss: 0.3693,  Recall: 1.0000, Precision: 0.5000, F2 Score: 0.8333, Accuracy: 0.5000
Epoch [5/50], Loss: 0.3399,  Recall: 0.9091, Precision: 0.5882, F2 Score: 0.8197, Accuracy: 0.6364
Epoch [6/50], Loss: 0.3108,  Recall: 0.7273, Precision: 0.6337, F2 Score: 0.7064, Accuracy: 0.6534
Epoch [7/50], Loss: 0.2909,  Recall: 0.6932, Precision: 0.7011, F2 Score: 0.6948, Accuracy: 0.6989
Epoch [8/50], Loss: 0.2600,  Recall: 0.8409, Precision: 0.6016, F2 Score: 0.7789, Accuracy: 0.6420
Epoch [9/50], Loss: 0.2391,  Recall: 0.7159, Precision: 0.6562, F2 Score: 0.7031, Accuracy: 0.6705
Epoch [10/50], Loss: 0.2202,  Recall: 0.8977, Precision: 0.6077, F2 Score: 0.8195, Accuracy: 0.6591
Epoch [11

In [ ]:
# Without scheduler
train_dataset_db1 = CustomDataset(X_static_train_db1, X_glu_train, X_weight_train, y_train_db1)
test_dataset_db1 = CustomDataset(X_static_test_db1, X_glu_test, X_weight_test, y_test_db1)

train_loader_db1 = DataLoader(train_dataset_db1, batch_size=32, shuffle=True)
test_loader_db1 = DataLoader(test_dataset_db1, batch_size=32, shuffle=False)

# Define the Multi-Input Model
class MultiInputModel(nn.Module):
    def __init__(self, static_input_dim, glu_input_dim, weight_input_dim):
        super(MultiInputModel, self).__init__()

        # Static feature branch
        self.static_fc = nn.Sequential(
            nn.Linear(static_input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for glu with attention and masking
        self.glu_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.glu_attention = nn.Sequential(
            nn.Linear(64, 1),  # Attention score for each time step
            nn.Softmax(dim=1)  # Normalize scores across valid time steps
        )
        self.glu_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for weight with attention and masking
        self.weight_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.weight_attention = nn.Sequential(
            nn.Linear(64, 1),
            nn.Softmax(dim=1)
        )
        self.weight_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Combined high-level features
        self.combined_fc = nn.Sequential(
            nn.Linear(32 + 32 + 32, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, static_input, glu_input, weight_input):
        # Static feature branch
        static_out = self.static_fc(static_input)

        # Glu branch with masking and attention
        glu_mask = (glu_input.sum(dim=-1) != 0).float()  # Mask for valid time steps
        glu_out, _ = self.glu_lstm(glu_input)  # Output shape: (batch_size, seq_len, hidden_size)

        attention_scores_glu = self.glu_attention(glu_out).squeeze(-1)  # Attention scores: (batch_size, seq_len)
        attention_scores_glu = attention_scores_glu * glu_mask  # Apply mask to attention scores
        attention_weights_glu = attention_scores_glu / (attention_scores_glu.sum(dim=1, keepdim=True) + 1e-8)  # Normalize

        glu_out = torch.bmm(attention_weights_glu.unsqueeze(1), glu_out).squeeze(1)  # Weighted sum: (batch_size, hidden_size)
        glu_out = self.glu_fc(glu_out)

        # Weight branch with masking and attention
        weight_mask = (weight_input.sum(dim=-1) != 0).float()  # Mask for valid time steps
        weight_out, _ = self.weight_lstm(weight_input)

        attention_scores_weight = self.weight_attention(weight_out).squeeze(-1)  # Attention scores: (batch_size, seq_len)
        attention_scores_weight = attention_scores_weight * weight_mask  # Apply mask to attention scores
        attention_weights_weight = attention_scores_weight / (attention_scores_weight.sum(dim=1, keepdim=True) + 1e-8)  # Normalize

        weight_out = torch.bmm(attention_weights_weight.unsqueeze(1), weight_out).squeeze(1)  # Weighted sum: (batch_size, hidden_size)
        weight_out = self.weight_fc(weight_out)

        # Combine all branches
        combined = torch.cat([static_out, glu_out, weight_out], dim=1)
        output = self.combined_fc(combined)
        return output


# weighted loss function
num_positive = target_db1.sum()
num_negative = len(target_db1) - num_positive
pos_weight_value = num_negative / num_positive
pos_weight = torch.tensor([pos_weight_value], dtype=torch.float32)

# Initialize the Model, Loss, and Optimizer
static_input_dim = X_static_train_db1.shape[1]
glu_input_dim = X_glu_train.shape[1]
weight_input_dim = X_weight_train.shape[1]

model = MultiInputModel(static_input_dim, glu_input_dim, weight_input_dim)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training Model
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=50, threshold=0.5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch in train_loader:
            static_input = batch["static"]
            glu_input = batch["glu"]
            weight_input = batch["weight"]
            target = batch["target"].unsqueeze(1)
            outputs = model(static_input, glu_input, weight_input)
            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        _, _, recall, precision, f2_score, accuracy = evaluate_model(model, test_loader, threshold)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, "
              f" Recall: {recall:.4f}, Precision: {precision:.4f}, "
              f"F2 Score: {f2_score:.4f}, Accuracy: {accuracy:.4f}")

def calculate_metrics(outputs, targets, threshold=0.5):
    predictions = (outputs > threshold).astype(int)
    targets = targets.astype(int)
    recall = recall_score(targets, predictions)
    precision = precision_score(targets, predictions)
    f2_score = fbeta_score(targets, predictions, beta=2)
    accuracy = accuracy_score(targets, predictions)
    return recall, precision, f2_score, accuracy

# Evaluation
def evaluate_model(model, test_loader, threshold=0.5):
    model.eval()
    all_outputs = []
    all_targets = []
    with torch.no_grad():
        for batch in test_loader:
            static_input = batch["static"]
            glu_input = batch["glu"]
            weight_input = batch["weight"]
            target = batch["target"]

            outputs = model(static_input, glu_input, weight_input)
            all_outputs.append(outputs.squeeze().numpy())
            all_targets.append(target.numpy())

    outputs = np.concatenate(all_outputs)
    targets = np.concatenate(all_targets)
    recall, precision, f2_score, accuracy = calculate_metrics(outputs, targets, threshold)
    return outputs, targets, recall, precision, f2_score, accuracy

train_model(model, train_loader_db1, test_loader_db1, criterion, optimizer)


Epoch [1/50], Loss: 0.6933,  Recall: 0.9432, Precision: 0.5390, F2 Score: 0.8202, Accuracy: 0.5682
Epoch [2/50], Loss: 0.6803,  Recall: 0.8068, Precision: 0.5966, F2 Score: 0.7537, Accuracy: 0.6307
Epoch [3/50], Loss: 0.6494,  Recall: 0.3523, Precision: 0.7750, F2 Score: 0.3954, Accuracy: 0.6250
Epoch [4/50], Loss: 0.6147,  Recall: 0.4432, Precision: 0.6610, F2 Score: 0.4745, Accuracy: 0.6080
Epoch [5/50], Loss: 0.5810,  Recall: 0.6932, Precision: 0.5922, F2 Score: 0.6703, Accuracy: 0.6080
Epoch [6/50], Loss: 0.5637,  Recall: 0.7159, Precision: 0.6300, F2 Score: 0.6969, Accuracy: 0.6477
Epoch [7/50], Loss: 0.5500,  Recall: 0.5568, Precision: 0.7313, F2 Score: 0.5847, Accuracy: 0.6761
Epoch [8/50], Loss: 0.5285,  Recall: 0.3750, Precision: 0.6735, F2 Score: 0.4115, Accuracy: 0.5966
Epoch [9/50], Loss: 0.5165,  Recall: 0.6023, Precision: 0.6709, F2 Score: 0.6148, Accuracy: 0.6534
Epoch [10/50], Loss: 0.5024,  Recall: 0.7045, Precision: 0.6739, F2 Score: 0.6982, Accuracy: 0.6818
Epoch [11

the model training for Dataset 2

In [ ]:
# weighted focus loss
from sklearn.metrics import recall_score, precision_score, fbeta_score, accuracy_score
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Dataset Class
class StaticDataset(Dataset):
    def __init__(self, static_data, target):
        if isinstance(static_data, pd.DataFrame):
            self.static_data = torch.tensor(static_data.to_numpy(), dtype=torch.float32)
        else:
            self.static_data = torch.tensor(static_data, dtype=torch.float32)
        self.target = torch.tensor(target, dtype=torch.float32)

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return {
            "static": self.static_data[idx],
            "target": self.target[idx],
        }


# Weighted Focal Loss
class WeightedFocalLoss(nn.Module):
    def __init__(self, pos_weight, gamma=1.5):
        super(WeightedFocalLoss, self).__init__()
        self.pos_weight = pos_weight
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.BCELoss(weight=(targets * self.pos_weight + (1 - targets)))(inputs, targets)
        pt = torch.exp(-BCE_loss)  # Probability of the correct class
        focal_loss = (1 - pt) ** self.gamma * BCE_loss
        return torch.mean(focal_loss)


# Static Model
class StaticModel(nn.Module):
    def __init__(self, static_input_dim):
        super(StaticModel, self).__init__()
        self.static_fc = nn.Sequential(
            nn.Linear(static_input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, static_input):
        return self.static_fc(static_input)


# Evaluation Function
def evaluate_model(model, test_loader, threshold=0.5):
    model.eval()
    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for batch in test_loader:
            static_input = batch["static"]
            target = batch["target"]

            outputs = model(static_input)
            all_outputs.append(outputs.squeeze().numpy())
            all_targets.append(target.numpy())

    outputs = np.concatenate(all_outputs)
    targets = np.concatenate(all_targets)

    # Metrics calculation
    predictions = (outputs > threshold).astype(int)
    recall = recall_score(targets, predictions)
    precision = precision_score(targets, predictions)
    f2_score = fbeta_score(targets, predictions, beta=2)
    accuracy = accuracy_score(targets, predictions)

    return outputs, targets, recall, precision, f2_score, accuracy


# Training Function
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=50, threshold=0.5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch in train_loader:
            static_input = batch["static"]
            target = batch["target"].unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(static_input).clamp(min=1e-7, max=1 - 1e-7)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Evaluate on the test set
        _, _, recall, precision, f2_score, accuracy = evaluate_model(model, test_loader, threshold)
        scheduler.step(f2_score)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, "
              f"Recall: {recall:.4f}, Precision: {precision:.4f}, "
              f"F2 Score: {f2_score:.4f}, Accuracy: {accuracy:.4f}")


# Initialize DataLoaders and Model
train_dataset_db2 = StaticDataset(X_static_train_db2, y_train_db2)
test_dataset_db2 = StaticDataset(X_static_test_db2, y_test_db2)

train_loader_db2 = DataLoader(train_dataset_db2, batch_size=32, shuffle=True)
test_loader_db2 = DataLoader(test_dataset_db2, batch_size=32, shuffle=False)

static_input_dim = X_static_train_db2.shape[1]
model2 = StaticModel(static_input_dim)

# Define Loss and Optimizer
num_positive = sum(y_train_db2)  # Positive samples
num_negative = len(y_train_db2) - num_positive  # Negative samples
pos_weight = num_negative / num_positive  # Weight for imbalance

criterion = WeightedFocalLoss(pos_weight=pos_weight, gamma=1.5)
optimizer = optim.Adam(model2.parameters(), lr=0.0005)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

# Train the Model
train_model(model2, train_loader_db2, test_loader_db2, criterion, optimizer, scheduler)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/50], Loss: 0.2277, Recall: 0.6047, Precision: 0.5843, F2 Score: 0.6005, Accuracy: 0.5872
Epoch [2/50], Loss: 0.1818, Recall: 0.7791, Precision: 0.6979, F2 Score: 0.7614, Accuracy: 0.7209
Epoch [3/50], Loss: 0.1376, Recall: 0.5233, Precision: 0.7759, F2 Score: 0.5597, Accuracy: 0.6860
Epoch [4/50], Loss: 0.1107, Recall: 0.5465, Precision: 0.7581, F2 Score: 0.5788, Accuracy: 0.6860
Epoch [5/50], Loss: 0.0925, Recall: 0.5814, Precision: 0.8333, F2 Score: 0.6188, Accuracy: 0.7326
Epoch [6/50], Loss: 0.0784, Recall: 0.6744, Precision: 0.7436, F2 Score: 0.6872, Accuracy: 0.7209
Epoch [7/50], Loss: 0.0641, Recall: 0.6977, Precision: 0.8108, F2 Score: 0.7177, Accuracy: 0.7674
Epoch [8/50], Loss: 0.0541, Recall: 0.8837, Precision: 0.6847, F2 Score: 0.8352, Accuracy: 0.7384
Epoch [9/50], Loss: 0.0414, Recall: 0.7442, Precision: 0.7619, F2 Score: 0.7477, Accuracy: 0.7558
Epoch [10/50], Loss: 0.0308, Recall: 0.7791, Precision: 0.7701, F2 Score: 0.7773, Accuracy: 0.7733
Epoch [11/50], Loss

In [ ]:
#dynamic threshold
from sklearn.metrics import precision_recall_curve, recall_score, precision_score, fbeta_score, accuracy_score
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
class StaticDataset(Dataset):
    def __init__(self, static_data, target):
        # Handle pandas DataFrame or NumPy array
        if isinstance(static_data, pd.DataFrame):
            self.static_data = torch.tensor(static_data.to_numpy(), dtype=torch.float32)
        else:
            self.static_data = torch.tensor(static_data, dtype=torch.float32)
        self.target = torch.tensor(target, dtype=torch.float32)  # Convert to tensor

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return {
            "static": self.static_data[idx],
            "target": self.target[idx],
        }

train_dataset_db2 = StaticDataset(X_static_train_db2, y_train_db2)
test_dataset_db2 = StaticDataset(X_static_test_db2, y_test_db2)

train_loader_db2 = DataLoader(train_dataset_db2, batch_size=32, shuffle=True)
test_loader_db2 = DataLoader(test_dataset_db2, batch_size=32, shuffle=False)

# Define the Static Model
class StaticModel(nn.Module):
    def __init__(self, static_input_dim):
        super(StaticModel, self).__init__()

        # Fully Connected Network for Static Features
        self.static_fc = nn.Sequential(
            nn.Linear(static_input_dim, 128),  # Increased neurons
            nn.BatchNorm1d(128),  # Batch Normalization
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),  # Batch Normalization
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),  # Batch Normalization
            nn.ReLU(),
            nn.Linear(32, 1),  # Final output layer
            nn.Sigmoid()
        )

    def forward(self, static_input):
        return self.static_fc(static_input)


# Initialize the Model, Loss, and Optimizer
static_input_dim = X_static_train_db2.shape[1]

model2 = StaticModel(static_input_dim)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model2.parameters(), lr=0.0005)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)


# Dynamic Threshold Finder
def find_best_threshold(outputs, targets, beta=2, min_threshold=0.1):
    precision, recall, thresholds = precision_recall_curve(targets, outputs)
    fbeta_scores = (1 + beta**2) * (precision[:-1] * recall[:-1]) / ((beta**2 * precision[:-1]) + recall[:-1] + 1e-5)

    valid_idx = thresholds >= min_threshold
    thresholds = thresholds[valid_idx]
    fbeta_scores = fbeta_scores[valid_idx]

    if len(fbeta_scores) == 0:
        return min_threshold, 0.0

    best_idx = np.argmax(fbeta_scores)
    return thresholds[best_idx], fbeta_scores[best_idx]


# Training Model
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=50):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch in train_loader:
            static_input = batch["static"]
            target = batch["target"].unsqueeze(1)

            # Forward pass
            outputs = model(static_input)
            loss = criterion(outputs, target)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Evaluate model and find the best threshold
        outputs, targets, best_threshold, recall, precision, f2_score, accuracy = evaluate_model(model, test_loader)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, "
              f"Best Threshold: {best_threshold:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, "
              f"F2 Score: {f2_score:.4f}, Accuracy: {accuracy:.4f}")

        # Step the scheduler
        scheduler.step(f2_score)


# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for batch in test_loader:
            static_input = batch["static"]
            target = batch["target"]

            outputs = model(static_input)
            all_outputs.append(outputs.squeeze().numpy())
            all_targets.append(target.numpy())

    outputs = np.concatenate(all_outputs)
    targets = np.concatenate(all_targets)

    # Find the best dynamic threshold
    best_threshold, best_fbeta = find_best_threshold(outputs, targets, beta=2)

    # Apply the best threshold
    predictions = (outputs > best_threshold).astype(int)
    recall = recall_score(targets, predictions)
    precision = precision_score(targets, predictions)
    f2_score = fbeta_score(targets, predictions, beta=2)
    accuracy = accuracy_score(targets, predictions)

    return outputs, targets, best_threshold, recall, precision, f2_score, accuracy


# Train the Static Model
train_model(model2, train_loader_db2, test_loader_db2, criterion, optimizer, scheduler)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/50], Loss: 0.6771, Best Threshold: 0.3908, Recall: 0.9651, Precision: 0.5425, F2 Score: 0.8350, Accuracy: 0.5756
Epoch [2/50], Loss: 0.5926, Best Threshold: 0.3362, Recall: 0.9419, Precision: 0.5870, F2 Score: 0.8402, Accuracy: 0.6395
Epoch [3/50], Loss: 0.5036, Best Threshold: 0.1483, Recall: 0.9767, Precision: 0.5600, F2 Score: 0.8502, Accuracy: 0.6047
Epoch [4/50], Loss: 0.4515, Best Threshold: 0.1052, Recall: 0.9535, Precision: 0.5734, F2 Score: 0.8419, Accuracy: 0.6221
Epoch [5/50], Loss: 0.4131, Best Threshold: 0.2046, Recall: 0.9767, Precision: 0.6000, F2 Score: 0.8678, Accuracy: 0.6628
Epoch [6/50], Loss: 0.3534, Best Threshold: 0.1162, Recall: 0.9535, Precision: 0.5857, F2 Score: 0.8471, Accuracy: 0.6395
Epoch [7/50], Loss: 0.3207, Best Threshold: 0.1515, Recall: 0.9767, Precision: 0.6043, F2 Score: 0.8696, Accuracy: 0.6686
Epoch [8/50], Loss: 0.3023, Best Threshold: 0.2704, Recall: 0.9535, Precision: 0.6357, F2 Score: 0.8668, Accuracy: 0.7035
Epoch [9/50], Loss: 0.27

In [ ]:
!gdown --id 13qUlcJCRuQ4SjuMp80UTzqRdWZ6N9FTe

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=13qUlcJCRuQ4SjuMp80UTzqRdWZ6N9FTe
To: /content/new_test_dataset_with_all_features.xlsx
100% 238k/238k [00:00<00:00, 42.6MB/s]


Fusion-WP

In [ ]:
from sklearn.metrics import precision_score, recall_score, fbeta_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import torch

# 加载测试数据
test_data = pd.read_excel("new_test_dataset_with_all_features.xlsx")

# 使用标准化参数对静态特征标准化
test_data[['年龄', '身高', '孕前BMI', '孕前体重']] = (test_data[['年龄', '身高', '孕前BMI', '孕前体重']] - mean_db1) / scale_db1

# 处理 glu 和 weight 特征中的缺失值
glu_features = test_data[['glu41', 'glu4b', 'glu4c', 'glu4d']].replace([' ', ''], np.nan).to_numpy()
weight_features = test_data[['weight0', 'weight1', 'weight2', 'weight3', 'weight4']].replace([' ', ''], np.nan).to_numpy()

glu_features = np.nan_to_num(glu_features.astype(np.float32), nan=0.0).reshape(len(glu_features), 4, 1)
weight_features = np.nan_to_num(weight_features.astype(np.float32), nan=0.0).reshape(len(weight_features), 5, 1)

# 准备模型1的静态特征
model_static_features = test_data[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层', '工作状态',
                                   '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重', '身高', '孕前BMI']].to_numpy()

# 使用模型2的标准化参数对静态特征进行标准化
test_data[['年龄', '身高', '孕前BMI', '孕前体重']] = (test_data[['年龄', '身高', '孕前BMI', '孕前体重']] - mean_db2) / scale_db2
model2_static_features = test_data[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层', '工作状态',
                                    '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重', '身高', '孕前BMI',
                                    '意外怀孕', '助孕治疗', '已生孩子数分层', 'ACE1', 'ACE2', 'ACE3',
                                    'ACE4', 'ACE5', 'ACE6', 'ACE7', 'ACE8', 'ACE9', 'ACE10', 'F1a',
                                    'F2a', 'F3a', 'F4a', 'F5a', 'F6a', 'F7a', 'F8a', 'F9a', 'F10a',
                                    'F11a', 'F12a', 'F13a', 'F14a', 'F15a', 'F16a', 'F17a', 'F18a',
                                    'F19a']].to_numpy()

# 模型1预测
model.eval()
with torch.no_grad():
    static_tensor = torch.tensor(model_static_features, dtype=torch.float32)
    glu_tensor = torch.tensor(glu_features, dtype=torch.float32)
    weight_tensor = torch.tensor(weight_features, dtype=torch.float32)
    outputs_model = model(static_tensor, glu_tensor, weight_tensor).squeeze().numpy()

predictions_model = (outputs_model > 0.5).astype(int)  # 使用固定阈值

# 模型2预测
model2.eval()
with torch.no_grad():
    static_tensor_model2 = torch.tensor(model2_static_features, dtype=torch.float32)
    outputs_model2 = model2(static_tensor_model2).squeeze().numpy()

predictions_model2 = (outputs_model2 > 0.5).astype(int)  # 使用固定阈值



final_predictions = []
for output1, output2 in zip(outputs_model, outputs_model2):
    # 根据加权概率确定最终预测
    weighted_output = 0.75 * output1 + 0.25 * output2  # 调整权重比例
    final_predictions.append(1 if weighted_output > 0.5 else 0)



# 假设 test_data 中有真实标签列 "target"
true_labels = test_data['target'].to_numpy()

# 计算性能指标
precision = precision_score(true_labels, final_predictions)
recall = recall_score(true_labels, final_predictions)
f2_score = fbeta_score(true_labels, final_predictions, beta=2)
accuracy = accuracy_score(true_labels, final_predictions)

# 打印结果
print("Precision:", precision)
print("Recall:", recall)
print("F2 Score:", f2_score)
print("Accuracy:", accuracy)


Precision: 0.5109489051094891
Recall: 0.8187134502923976
F2 Score: 0.7306889352818372
Accuracy: 0.8012048192771084


<ipython-input-24-96d6b5eae3ba>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  glu_features = test_data[['glu41', 'glu4b', 'glu4c', 'glu4d']].replace([' ', ''], np.nan).to_numpy()
<ipython-input-24-96d6b5eae3ba>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  weight_features = test_data[['weight0', 'weight1', 'weight2', 'weight3', 'weight4']].replace([' ', ''], np.nan).to_numpy()


Fusion-CIL

In [ ]:
from sklearn.metrics import precision_score, recall_score, fbeta_score, accuracy_score
from sklearn.preprocessing import StandardScaler
test_data = pd.read_excel("new_test_dataset_with_all_features.xlsx")
# 手动输入的动态阈值
model_best_threshold = 0.1190  # 训练好的模型1动态阈值
model2_best_threshold = 0.2396  # 训练好的模型2动态阈值


# Standardize static features using saved means and scales
test_data[['年龄', '身高', '孕前BMI', '孕前体重']] = (test_data[['年龄', '身高', '孕前BMI', '孕前体重']] - mean_db1) / scale_db1

# Handle missing values in glu and weight features
glu_features = test_data[['glu41', 'glu4b', 'glu4c', 'glu4d']].replace([' ', ''], np.nan).to_numpy()
weight_features = test_data[['weight0', 'weight1', 'weight2', 'weight3', 'weight4']].replace([' ', ''], np.nan).to_numpy()

glu_features = np.nan_to_num(glu_features.astype(np.float32), nan=0.0).reshape(len(glu_features), 4, 1)
weight_features = np.nan_to_num(weight_features.astype(np.float32), nan=0.0).reshape(len(weight_features), 5, 1)

# Prepare static features for model
model_static_features = test_data[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层', '工作状态',
                                  '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重', '身高', '孕前BMI']].to_numpy()

# Standardize static features for model2 using its saved parameters
test_data[['年龄', '身高', '孕前BMI', '孕前体重']] = (test_data[['年龄', '身高', '孕前BMI', '孕前体重']] - mean_db2) / scale_db2
model2_static_features = test_data[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层', '工作状态',
                                    '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重', '身高', '孕前BMI',
                                    '意外怀孕', '助孕治疗', '已生孩子数分层', 'ACE1', 'ACE2', 'ACE3',
                                    'ACE4', 'ACE5', 'ACE6', 'ACE7', 'ACE8', 'ACE9', 'ACE10', 'F1a',
                                    'F2a', 'F3a', 'F4a', 'F5a', 'F6a', 'F7a', 'F8a', 'F9a', 'F10a',
                                    'F11a', 'F12a', 'F13a', 'F14a', 'F15a', 'F16a', 'F17a', 'F18a',
                                    'F19a']].to_numpy()
# 用模型1预测
model.eval()
with torch.no_grad():
    static_tensor = torch.tensor(model_static_features, dtype=torch.float32)
    glu_tensor = torch.tensor(glu_features, dtype=torch.float32)
    weight_tensor = torch.tensor(weight_features, dtype=torch.float32)
    outputs_model = model(static_tensor, glu_tensor, weight_tensor).squeeze().numpy()

predictions_model = (outputs_model > model_best_threshold).astype(int)

# 用模型2预测
model2.eval()
with torch.no_grad():
    static_tensor_model2 = torch.tensor(model2_static_features, dtype=torch.float32)
    outputs_model2 = model2(static_tensor_model2).squeeze().numpy()

predictions_model2 = (outputs_model2 > model2_best_threshold).astype(int)

# 最终预测
final_predictions = []
for pred1, pred2, output1, output2 in zip(predictions_model, predictions_model2, outputs_model, outputs_model2):
    if pred1 == pred2:
        final_predictions.append(pred1)
    else:
        # 使用置信区间更高的预测值
        if abs(output1 - model_best_threshold) > abs(output2 - model2_best_threshold):
            final_predictions.append(pred1)
        else:
            final_predictions.append(pred2)


# 假设 test_data 中有真实标签列 "真实标签"
true_labels = test_data['target'].to_numpy()

# 计算性能指标
precision = precision_score(true_labels, final_predictions)
recall = recall_score(true_labels, final_predictions)
f2_score = fbeta_score(true_labels, final_predictions, beta=2)
accuracy = accuracy_score(true_labels, final_predictions)

# 打印结果
print("Precision:", precision)
print("Recall:", recall)
print("F2 Score:", f2_score)
print("Accuracy:", accuracy)
print("Model 1 Best Threshold:", model_best_threshold)
print("Model 2 Best Threshold:", model2_best_threshold)








<ipython-input-9-ee78a2c19fec>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  glu_features = test_data[['glu41', 'glu4b', 'glu4c', 'glu4d']].replace([' ', ''], np.nan).to_numpy()
<ipython-input-9-ee78a2c19fec>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  weight_features = test_data[['weight0', 'weight1', 'weight2', 'weight3', 'weight4']].replace([' ', ''], np.nan).to_numpy()


Precision: 0.8553459119496856
Recall: 0.7953216374269005
F2 Score: 0.8066429418742586
Accuracy: 0.9301204819277108
Model 1 Best Threshold: 0.119
Model 2 Best Threshold: 0.2396
